In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
api_key = "TGV2OZgIKqiCEWvhVUAUDGBB99Eixo1z2ILlZ0h3lRdutfklTwRlqzPYpIazsYcn"
secret_key = "CrgLtKo6MRxdL6tJRNKokKPKOAWa0Rn59Q9KfX57yal0E1WTqz79jk2IT03g6VcZ"
#test api_key
test_api_Key = "ni1RGarrqKhXFMnmqZt2EEbEJkpGDXs4AxHml3r7aevzg7uqUR1rTFbNngWiTOln"
test_secret_Key = "HqFm7zQNQPOEJdYMgLho7engJbah6FAH98Eqf607h467CB9RM9e6dHDytnQbKGgk"

In [3]:
client = Client(api_key = test_api_Key, api_secret = test_secret_Key, tld = "com", testnet = True)

In [8]:
class LongOnlyTrader(): # optimized for TWM
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = [] 
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
            self.twm.join() # required in script!
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # stop trading session
        if event_time >= datetime(2024, 3, 27, 8, 25):
            self.twm.stop()
            if self.position != 0:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
            else: 
                print("STOP")
        
        #more stop examples:
        # if self.trades >= xyz
        # if self.cum_profits <> xyz
        
        else:    
            # print out
            print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
            # feed df (add new bar / update latest bar)
            self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
            # prepare features and define strategy/trading positions whenever the latest bar is complete
            if complete == True:
                self.define_strategy()
                self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [5]:
import pytz
# 현재 시간을 세계 협정시로 변환
utc_now = datetime.utcnow().replace(tzinfo=pytz.utc)
print("현재 세계 협정시:", utc_now)

현재 세계 협정시: 2024-03-27 08:19:05.685596+00:00


In [9]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1711526160149,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.02000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '8602.31691910', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.00000000

In [10]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3,3]
units = 0.01
position = 0

In [11]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                       volume_thresh = volume_thresh, units = units, position = position)

In [13]:
trader.start_trading(historical_days = 1/24)

......

----------------------------------------------------------------------------------------------------
2024-03-27 08:23:00.137000 | GOING LONG
2024-03-27 08:23:00.137000 | Base_Units = 0.01 | Quote_Units = 699.86356 | Price = 69986.356 
2024-03-27 08:23:00.137000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

............................

----------------------------------------------------------------------------------------------------
2024-03-27 08:24:00.177000 | GOING NEUTRAL
2024-03-27 08:24:00.177000 | Base_Units = 0.01 | Quote_Units = 699.0800093 | Price = 69908.00093 
2024-03-27 08:24:00.177000 | Profit = -0.784 | CumProfits = -0.784 
----------------------------------------------------------------------------------------------------

...........................STOP


In [15]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1711527840177,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.02000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '8601.53336840', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.00000000

In [17]:
trader.cum_profits #usdt

-0.784

In [18]:
np.sum(trader.trade_values)

-0.783550699999978